In [9]:
import os
import cv2
import random
import numpy as np 
import pandas as pd 
import seaborn as sns
from tqdm import tqdm

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# To plot pretty figures
%matplotlib inline
import matplotlib as mlp
import matplotlib.pyplot as plt

## Data Loading

In [10]:
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
train_img = [] #contains the images used for training the model
test_img = []
train_labels = [] #label of each image in x_train 
test_labels = []
TRAIN_PATH = '../input/brain-tumor-classification-mri/Training'
TEST_PATH = '../input/brain-tumor-classification-mri/Testing'
new_size = (255, 255)

for label in labels:
    img_dir = os.path.join(TRAIN_PATH, label)
    for img_file in os.listdir(img_dir):
        img = cv2.imread(f'{img_dir}/{img_file}')
        img = cv2.resize(img, new_size)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)/255
        train_img.append(img)
        train_labels.append(label)
        
train_img = np.stack(train_img)
train_labels = np.stack(train_labels)

print("train_img shape : ", train_img.shape)
print("train_labels shape : ", train_labels.shape)

for label in labels:
    img_dir = os.path.join(TEST_PATH, label)
    for img_file in os.listdir(img_dir):
        img = cv2.imread(f'{img_dir}/{img_file}')
        img = cv2.resize(img, new_size)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)/255
        test_img.append(img)
        test_labels.append(label)
        
test_img = np.stack(test_img)
test_labels = np.stack(test_labels)

print("test_img shape : ", test_img.shape)
print("test_labels shape : ", test_labels.shape)

class_map = {
    'no_tumor': 0,
    'glioma_tumor': 1,
    'pituitary_tumor': 2,
    'meningioma_tumor': 3
}

train_labels = np.array([class_map[label] for label in train_labels])
test_labels = np.array([class_map[label] for label in test_labels])

train_img shape :  (2870, 255, 255)
train_labels shape :  (2870,)
test_img shape :  (394, 255, 255)
test_labels shape :  (394,)


# Pre-processing

In [11]:
train_img = train_img.reshape(2870, 255, 255, 1)
test_img = test_img.reshape(394, 255, 255, 1)

# Convolutional Multilayer Perceptron

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_validate, cross_val_score

In [13]:
import keras 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten 
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K 

### Create the model

In [14]:
model = Sequential() 

model.add(Conv2D(25, kernel_size = 5, activation='relu', input_shape=(255, 255, 1), strides = 1))
model.add(AveragePooling2D(pool_size = 10, strides = 2))

model.add(Conv2D(10, kernel_size = 5, activation='relu', strides = 1))
model.add(AveragePooling2D(pool_size = 5, strides = 2))

model.add(Conv2D(8, kernel_size = 2, activation='relu', strides = 1))
model.add(AveragePooling2D(pool_size = 2, strides = 2))

model.add(Flatten())
model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 251, 251, 25)      650       
_________________________________________________________________
average_pooling2d_3 (Average (None, 121, 121, 25)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 117, 117, 10)      6260      
_________________________________________________________________
average_pooling2d_4 (Average (None, 57, 57, 10)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 8)         328       
_________________________________________________________________
average_pooling2d_5 (Average (None, 28, 28, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)             


### Compile the model

In [15]:
# model.compile(loss = keras.losses.categorical_crossentropy, optimizer = keras.optimizers.adadelta_v2.Adadelta(), metrics = ['accuracy'])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [16]:
model.fit(train_img, train_labels, validation_data = (test_img, test_labels), epochs = 10)

Epoch 1/10
90/90 [==============================] - 55s 608ms/step - loss: 1.0985 - accuracy: 0.5258 - val_loss: 1.8927 - val_accuracy: 0.2893
Epoch 2/10
90/90 [==============================] - 54s 603ms/step - loss: 0.8530 - accuracy: 0.6456 - val_loss: 2.3986 - val_accuracy: 0.3046
Epoch 3/10
90/90 [==============================] - 54s 604ms/step - loss: 0.6782 - accuracy: 0.7282 - val_loss: 2.3786 - val_accuracy: 0.4315
Epoch 4/10
90/90 [==============================] - 55s 607ms/step - loss: 0.6023 - accuracy: 0.7631 - val_loss: 2.8075 - val_accuracy: 0.4594
Epoch 5/10
90/90 [==============================] - 55s 608ms/step - loss: 0.5752 - accuracy: 0.7829 - val_loss: 2.4305 - val_accuracy: 0.5406
Epoch 6/10
90/90 [==============================] - 55s 607ms/step - loss: 0.5041 - accuracy: 0.8052 - val_loss: 2.5017 - val_accuracy: 0.5609
Epoch 7/10
90/90 [==============================] - 54s 603ms/step - loss: 0.4117 - accuracy: 0.8456 - val_loss: 3.1695 - val_accuracy: 0.5102

### Evaluate the model

In [17]:
score = model.evaluate(test_img, test_labels, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

y_prob = model.predict(test_img)
y_pred = np.argmax(y_prob, axis = 1)
print(y_pred)

Test loss: 3.3895232677459717
Test accuracy: 0.6497461795806885
[0 0 0 3 3 3 3 3 0 0 0 3 3 0 0 0 2 2 3 0 0 1 1 0 3 3 3 1 1 0 3 2 3 0 3 3 0
 0 0 3 0 3 3 0 0 0 3 3 0 3 0 3 3 1 3 3 3 0 1 3 1 3 3 1 2 3 1 1 3 0 3 3 3 0
 0 3 3 1 3 2 0 3 3 0 3 3 3 0 3 3 2 3 3 3 1 0 3 0 0 0 3 3 0 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 3 0 3 3 0 0
 0 0 0 2 0 0 0 0 0 0 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0 0 0 0 0 3 0 3
 0 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 2 2 2 2 2 3 2 3 3 2 2
 2 3 2 2 3 3 3 3 2 2 3 2 2 2 2 2 3 2 2 2 3 2 3 3 2 3 3 2 2 3 3 3 2 2 2 2 2
 2 2 3 2 3 2 2 3 3 2 3 2 3 3 2 3 2 2 3 2 2 2 2 3]


In [18]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, y_pred, zero_division=0))

precision = precision_score(test_labels, y_pred, average = "weighted", zero_division = 0)
recall = recall_score(test_labels, y_pred, average = "weighted", zero_division = 0)
f1 = f1_score(test_labels, y_pred, average = "weighted", zero_division = 0)
roc = roc_auc_score(test_labels, y_prob, average = "weighted", multi_class="ovr")
print("roc = ", roc)

              precision    recall  f1-score   support

           0       0.70      0.84      0.76       105
           1       1.00      0.12      0.21       100
           2       0.87      0.61      0.71        74
           3       0.54      0.97      0.70       115

    accuracy                           0.65       394
   macro avg       0.78      0.63      0.60       394
weighted avg       0.76      0.65      0.59       394

roc =  0.8239758415145373


Even worse than random classifier. Never predicts class 0 and class 1. When it should predict class 2, it does though. Somehow the less predominant class is the most predicted.

## Architecture 2

In [19]:
model = Sequential() 

model.add(Conv2D(25, kernel_size = 3, activation='relu', input_shape=(255, 255, 1), strides = 1))
model.add(AveragePooling2D(pool_size = 5, strides = 2))

model.add(Conv2D(10, kernel_size = 3, activation='relu', strides = 1))
model.add(AveragePooling2D(pool_size = 5, strides = 2))

model.add(Conv2D(8, kernel_size = 2, activation='relu', strides = 1))
model.add(AveragePooling2D(pool_size = 2, strides = 2))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 253, 253, 25)      250       
_________________________________________________________________
average_pooling2d_6 (Average (None, 125, 125, 25)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 123, 123, 10)      2260      
_________________________________________________________________
average_pooling2d_7 (Average (None, 60, 60, 10)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 59, 59, 8)         328       
_________________________________________________________________
average_pooling2d_8 (Average (None, 29, 29, 8)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6728)             

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_img, train_labels, validation_data = (test_img, test_labels), epochs = 10)

Epoch 1/10
90/90 [==============================] - 29s 314ms/step - loss: 1.1892 - accuracy: 0.4589 - val_loss: 1.8714 - val_accuracy: 0.2310
Epoch 2/10
90/90 [==============================] - 27s 301ms/step - loss: 0.9043 - accuracy: 0.6216 - val_loss: 1.6729 - val_accuracy: 0.3909
Epoch 3/10
90/90 [==============================] - 28s 308ms/step - loss: 0.7520 - accuracy: 0.6944 - val_loss: 1.5534 - val_accuracy: 0.3858
Epoch 4/10
90/90 [==============================] - 28s 309ms/step - loss: 0.6648 - accuracy: 0.7289 - val_loss: 1.6312 - val_accuracy: 0.4772
Epoch 5/10
90/90 [==============================] - 28s 311ms/step - loss: 0.5651 - accuracy: 0.7847 - val_loss: 1.7495 - val_accuracy: 0.4619
Epoch 6/10
90/90 [==============================] - 28s 312ms/step - loss: 0.5002 - accuracy: 0.8024 - val_loss: 1.8391 - val_accuracy: 0.4391
Epoch 7/10
90/90 [==============================] - 28s 311ms/step - loss: 0.4158 - accuracy: 0.8456 - val_loss: 2.1332 - val_accuracy: 0.5152

In [21]:
score = model.evaluate(test_img, test_labels, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

y_prob = model.predict(test_img)
y_pred = np.argmax(y_prob, axis = 1)
print(y_pred)

print(classification_report(test_labels, y_pred, zero_division=0))

precision = precision_score(test_labels, y_pred, average = "weighted", zero_division = 0)
recall = recall_score(test_labels, y_pred, average = "weighted", zero_division = 0)
f1 = f1_score(test_labels, y_pred, average = "weighted", zero_division = 0)
roc = roc_auc_score(test_labels, y_prob, average = "weighted", multi_class="ovr")
print("roc = ", roc)

Test loss: 2.997210741043091
Test accuracy: 0.4796954393386841
[0 0 0 3 1 1 0 3 0 0 3 0 3 0 0 0 2 2 3 0 3 1 1 3 0 1 0 1 1 0 0 3 3 0 0 0 0
 0 0 0 0 3 3 0 0 0 3 0 0 1 0 0 0 1 1 1 0 2 3 3 1 3 0 3 3 0 1 1 0 0 3 3 0 0
 3 1 3 1 0 0 3 0 0 3 3 3 3 0 3 0 3 3 0 3 1 3 0 0 0 0 3 3 0 3 0 0 0 0 0 3 3
 3 1 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 0 0 0 3 0 3 0 3 3 0 0 0 3 3 3 3 3 3 3
 3 3 0 0 0 0 0 3 0 3 0 0 0 3 3 0 0 0 3 0 0 0 3 0 3 3 3 3 0 0 3 0 0 0 0 0 3
 0 3 3 0 3 3 3 0 3 0 3 3 0 3 3 0 0 0 3 0 3 0 0 0 3 3 0 0 0 3 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 3 1 0 3 0 0 3 0 0 2 0 0 0 0 0 0 0 3 0 0 0 0 0 3 0 0 0
 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 3 3 2 0
 0 1 2 2 0 2 3 2 2 2 0 2 0 2 0 0 0 3 0 2 2 0 3 3 3 3 0 0 0 2 3 0 0 2 0 3 0
 0 2 2 3 0 2 0 1 2 0 0 0 0 2 2 0 3 2 0 2 0 3 2 0]
              precision    recall  f1-score   support

           0       0.40      0.90      0.55       105
           1       0.81     

## Architecture 3

In [22]:
model = Sequential() 

model.add(Conv2D(35, kernel_size = 3, activation='relu', input_shape=(255, 255, 1), strides = 1))
model.add(AveragePooling2D(pool_size = 3, strides = 1))

model.add(Conv2D(20, kernel_size = 3, activation='relu', strides = 1))
model.add(AveragePooling2D(pool_size = 3, strides = 1))

model.add(Conv2D(10, kernel_size = 2, activation='relu', strides = 1))
model.add(AveragePooling2D(pool_size = 2, strides = 2))

model.add(Conv2D(8, kernel_size = 2, activation='relu', strides = 1))
model.add(AveragePooling2D(pool_size = 2, strides = 2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 253, 253, 35)      350       
_________________________________________________________________
average_pooling2d_9 (Average (None, 251, 251, 35)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 249, 249, 20)      6320      
_________________________________________________________________
average_pooling2d_10 (Averag (None, 247, 247, 20)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 246, 246, 10)      810       
_________________________________________________________________
average_pooling2d_11 (Averag (None, 123, 123, 10)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 122, 122, 8)      

In [23]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_img, train_labels, validation_data = (test_img, test_labels), epochs = 5)

Epoch 1/5
90/90 [==============================] - 167s 2s/step - loss: 1.1114 - accuracy: 0.5202 - val_loss: 1.6840 - val_accuracy: 0.3528
Epoch 2/5
90/90 [==============================] - 166s 2s/step - loss: 0.7149 - accuracy: 0.7063 - val_loss: 2.0456 - val_accuracy: 0.5000
Epoch 3/5
90/90 [==============================] - 165s 2s/step - loss: 0.4624 - accuracy: 0.8105 - val_loss: 1.8276 - val_accuracy: 0.5787
Epoch 4/5
90/90 [==============================] - 166s 2s/step - loss: 0.2846 - accuracy: 0.8920 - val_loss: 2.0302 - val_accuracy: 0.6904
Epoch 5/5
90/90 [==============================] - 165s 2s/step - loss: 0.1648 - accuracy: 0.9397 - val_loss: 2.6422 - val_accuracy: 0.6929


In [24]:
score = model.evaluate(test_img, test_labels, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

y_prob = model.predict(test_img)
y_pred = np.argmax(y_prob, axis = 1)
print(y_pred)

print(classification_report(test_labels, y_pred, zero_division=0))

precision = precision_score(test_labels, y_pred, average = "weighted", zero_division = 0)
recall = recall_score(test_labels, y_pred, average = "weighted", zero_division = 0)
f1 = f1_score(test_labels, y_pred, average = "weighted", zero_division = 0)
roc = roc_auc_score(test_labels, y_prob, average = "weighted", multi_class="ovr")
print("roc = ", roc)

Test loss: 2.642218828201294
Test accuracy: 0.692893385887146
[0 0 0 3 3 1 3 3 0 0 0 0 3 0 0 0 2 0 3 0 0 1 0 0 2 1 2 1 1 0 2 2 3 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 1 3 1 0 0 0 3 1 3 0 0 2 0 1 1 0 0 3 3 0 0
 0 1 3 1 0 3 0 3 3 0 0 1 0 0 1 0 3 3 3 3 1 0 0 0 0 0 0 3 3 3 3 3 3 3 3 3 3
 0 3 3 3 3 0 3 3 3 3 3 0 3 3 3 0 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 0 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3 0 3 3 3 3
 3 3 3 0 3 0 3 0 3 0 3 3 3 3 3 3 3 3 3 3 3 0 3 3 0 3 3 3 3 3 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 3 2 3 3 2 2
 2 0 2 2 2 0 2 3 2 2 2 2 2 2 2 2 2 2 2 2 0 0 3 3 3 3 2 2 2 2 3 2 2 2 2 2 2
 2 2 0 2 2 2 0 0 0 0 2 2 2 0 2 2 3 2 2 2 2 2 2 2]
              precision    recall  f1-score   support

           0       0.56      1.00      0.72       105
           1       0.94      